# Example with spikeinterface NEW API

This is not based on actual spikeinterfacre but the futur one!!!

For installing, you need to do this in a fresh environement:
```
git clone https://github.com/samuelgarcia/spikeinterface.git
cd spikeinterface
python setup.py develop
```





In [1]:
# Import packages    
%matplotlib widget
import matplotlib.pyplot as plt
from pathlib import Path
import spikeinterface.full as si
from probeinterface import read_spikeglx
from probeinterface.plotting import plot_probe
import numpy as np

In [2]:
print(si.installed_sorters())

['herdingspikes', 'spykingcircus', 'tridesclous']


## read dataset

In [3]:
base_folder = Path('/home/arthur/Documents/SpikeSorting/Test_20210518/') 

data_folder = base_folder / 'raw_awake'

out_path = Path('/media/storage/spikesorting_output/SONG/')

recording = si.SpikeGLXRecordingExtractor(data_folder)
print(recording)

probe = read_spikeglx(data_folder / 'raw_awake_01_g0_t0.imec0.ap.meta')
print(probe)
recording = recording.set_probe(probe)
print(recording)

SpikeGLXRecordingExtractor: 385 channels - 1 segments - 30.0kHz - 3881.358s
384
Probe - 384ch
ChannelSliceRecording: 384 channels - 1 segments - 30.0kHz - 3881.358s


In [4]:
fs = recording.get_sampling_frequency()

sing_window = (1000, 1100) # Time in seconds

sing_rec = recording.frame_slice(sing_window[0]*fs, sing_window[1]*fs)
sing_rec = sing_rec.set_probe(probe)
print(sing_rec)

FrameSliceRecording: 384 channels - 1 segments - 30.0kHz - 100.000s


In [5]:
ids = sing_rec.channel_ids[0:10]
w_ts = si.plot_timeseries(sing_rec, time_range=(0,50), channel_ids=ids) # plotting just the first 10 channels
w_ts.figure.suptitle('Recording First 10 Channels')
w_ts.ax.set_ylabel('Channels IDs')

ticks = list(np.linspace(int(w_ts.ax.get_ylim()[0]), int(w_ts.ax.get_ylim()[1]), len(ids)+2))
w_ts.ax.set_yticks(ticks[1:-1])
w_ts.ax.set_yticklabels(ids)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0, 0.4545454545454959, 'imec0.ap#AP0'),
 Text(0, 565.909090909091, 'imec0.ap#AP1'),
 Text(0, 1131.3636363636365, 'imec0.ap#AP2'),
 Text(0, 1696.818181818182, 'imec0.ap#AP3'),
 Text(0, 2262.2727272727275, 'imec0.ap#AP4'),
 Text(0, 2827.727272727273, 'imec0.ap#AP5'),
 Text(0, 3393.1818181818185, 'imec0.ap#AP6'),
 Text(0, 3958.636363636364, 'imec0.ap#AP7'),
 Text(0, 4524.09090909091, 'imec0.ap#AP8'),
 Text(0, 5089.545454545455, 'imec0.ap#AP9')]

## check probe

In [6]:
si.plot_probe_map(sing_rec, channel_ids=sing_rec.channel_ids[0:100]) # You can change which channels are plotted

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## run tridesclous

This is slow, run this once.

In [7]:
tdc_params = {'freq_min': 300.,
              'freq_max': 6000.,
              'detect_threshold' : 6, 
              'common_ref_removal': True, 
              'nested_params' : {'peak_detector': {'adjacency_radius_um': 100},
                                 'clean_peaks': {'alien_value_threshold': 100.}}
                            }
sorting_TDC = si.run_tridesclous(sing_rec, output_folder=out_path / 'output_TDC_new',
         verbose=True, **tdc_params)

Local copy of recording


write_binary_recording: 100%|██████████| 37/37 [00:05<00:00,  6.88it/s]


DataIO <id: 139752677783680> 
  workdir: /media/storage/spikesorting_output/SONG/output_TDC_new
  sample_rate: 30000.482083
  total_channel: 384
  channel_groups: 0 [ch0 ch1 ch2 ch3 ... ch380 ch381 ch382 ch383]
  nb_segment: 1
  length: 3000048
  durations: 100.0 s.
catalogue_nested_params
{'chunksize': 30000,
 'clean_cluster': {'apply_auto_merge_cluster': True,
                   'apply_auto_split': True,
                   'apply_trash_low_extremum': True,
                   'apply_trash_not_aligned': True,
                   'apply_trash_small_cluster': True},
 'clean_peaks': {'alien_value_threshold': 100.0, 'mode': 'extremum_amplitude'},
 'cluster_kargs': {'adjacency_radius_um': 50.0,
                   'high_adjacency_radius_um': 30.0,
                   'max_loop': 7680,
                   'min_cluster_size': 20},
 'cluster_method': 'pruningshears',
 'duration': 99.99999305677824,
 'extract_waveforms': {'wf_left_long_ms': -2.5,
                       'wf_left_ms': -1.0,
         

## run herdingspikes  -- not working yet

This is slow, run this once.

In [ ]:
hs_params = {'clustering_bandwidth': 5.5,
             'clustering_alpha': 5.5, 
             'clustering_n_jobs': -1, 
             'clustering_bin_seeding': True, 
             'clustering_min_bin_freq': 16,
             'clustering_subset': None, 
             'left_cutout_time': 0.3, 
             'right_cutout_time': 1.8,
             'detect_threshold': 6, # default is 20
             'probe_masked_channels': [],
             'probe_inner_radius': 70, 
             'probe_neighbor_radius': 90, 
             'probe_event_length': 0.26,
             'probe_peak_jitter': 0.2, 
             't_inc': 100000, 
             'num_com_centers': 1, 
             'maa': 12, 
             'ahpthr': 11, 
             'out_file_name': 'HS2_detected', 
             'decay_filtering': False, 
             'save_all': False, 
             'amp_evaluation_time': 0.4, 
             'spk_evaluation_time': 1.0, 
             'pca_ncomponents': 2, 
             'pca_whiten': True, 
             'freq_min': 300.0, 
             'freq_max': 6000.0, 
             'filter': True, 
             'pre_scale': True, 
             'pre_scale_value': 20.0, 
             'filter_duplicates': True}

sorting_HS = si.run_herdingspikes(sing_rec, output_folder=out_path / 'output_herdingspikes', **hs_params)
print(sorting_HS)

## run spyking circus

This is slow, run this once.

In [ ]:
sc_params = {'detect_sign': -1,
             'adjacency_radius': 100,
             'detect_threshold': 6,
             'template_width_ms': 3,
             'filter': True,
             'merge_spikes': True,
             'auto_merge': 0.75,
             'num_workers': None,
             'whitening_max_elts': 1000,
             'clustering_max_elts': 10000}
sorting_SC = si.run_spykingcircus(sing_rec, output_folder=out_path / 'output_spykingcircus_new', **sc_params)

## Sorting results

In [8]:
#sorting_HS = si.HerdingspikesSortingExtractor(out_path / 'output_herdingspikes')
#print(sorting_HS)

sorting_SC = si.SpykingCircusSortingExtractor(out_path / 'output_spykingcircus_new')
print(sorting_SC)

sorting_TDC = si.TridesclousSortingExtractor(out_path / 'output_TDC_new')
print(sorting_TDC)

SpykingCircusSortingExtractor: 192 units - 1 segments - 30.0kHz
TridesclousSortingExtractor: 6 units - 1 segments - 30.0kHz


## save sorting in an other formats

In [9]:
save_folder = out_path / 'save_sorting'
save_folder.mkdir(exist_ok=True)

In [10]:
#si.NpzSortingExtractor.write_sorting(sorting_HS, save_folder / 'HS.npz')
si.NpzSortingExtractor.write_sorting(sorting_SC, save_folder / 'SC_new.npz')
si.NpzSortingExtractor.write_sorting(sorting_TDC, save_folder / 'TDCS3_new.npz')

## Reimporting .npz

In [18]:
TDC = si.NpzSortingExtractor(save_folder / 'TDCS3_new.npz')
SC = si.NpzSortingExtractor(save_folder / 'SC_new.npz')

In [19]:
print(TDC)
print(SC)

NpzSortingExtractor: 6 units - 1 segments - 30.0kHz
  file_path: /media/storage/spikesorting_output/SONG/save_sorting/TDCS3_new.npz
NpzSortingExtractor: 192 units - 1 segments - 30.0kHz
  file_path: /media/storage/spikesorting_output/SONG/save_sorting/SC_new.npz


## Extracting waveforms to work with Phy Exporter

In [13]:
sing_rec_bp = si.bandpass_filter(sing_rec, freq_min=300, freq_max=6000) # apply filter
folder_TDC = out_path / 'Phy/TDC-SONG'
folder_TDC_wv = folder_TDC / 'waveforms_filtered_new'
we_TDC = si.extract_waveforms(sing_rec_bp, TDC, folder_TDC_wv,
    load_if_exists=True,
    ms_before=1, ms_after=2., max_spikes_per_unit=500,
    chunk_size=30000, n_jobs=6, progress_bar=True)
print(we_TDC)

extract waveforms: 100%|██████████| 101/101 [00:07<00:00, 13.82it/s]

WaveformExtractor: 384 channels - 6 units - 1 segments
  before:30 after60 n_per_units: 500


In [14]:
folder_TDC_params = out_path / 'Phy/TDC-SONG/phy-params_new'
si.export_to_phy(sing_rec_bp, TDC, folder_TDC_params, waveform_extractor=we_TDC,
            compute_pc_features=False, compute_amplitudes=True,
            remove_if_exists=True, chunk_size=30000, n_jobs=6, progress_bar=True)

You can run spikeinterface.toolkit.preprocessing.bandpass_filter(recording)
write_binary_recording with n_jobs 6  chunk_size 30000


extract amplitudes: 100%|██████████| 101/101 [00:07<00:00, 14.34it/s]

Run:
phy template-gui  /media/storage/spikesorting_output/SONG/Phy/TDC-SONG/phy-params_new/params.py


In [20]:
folder_SC = out_path / 'Phy/SC-SONG'
folder_SC_wv = folder_SC / 'waveforms_filtered_new'
we_SC = si.extract_waveforms(sing_rec_bp, SC, folder_SC_wv,
    load_if_exists=True, ms_before=1, ms_after=2., max_spikes_per_unit=500,
    n_jobs=6, chunk_size=30000, progress_bar=True)
print(we_SC)

extract waveforms: 100%|██████████| 101/101 [00:13<00:00,  7.57it/s]

WaveformExtractor: 384 channels - 192 units - 1 segments
  before:30 after60 n_per_units: 500


In [21]:
folder_SC_params = out_path / 'Phy/SC-SONG/phy-params_new'
si.export_to_phy(sing_rec_bp, SC, folder_SC_params, waveform_extractor=we_SC,
            compute_pc_features=False, compute_amplitudes=True,
            remove_if_exists=True, chunk_size=30000, n_jobs=6, progress_bar=True)

You can run spikeinterface.toolkit.preprocessing.bandpass_filter(recording)
write_binary_recording with n_jobs 6  chunk_size 30000


extract amplitudes: 100%|██████████| 101/101 [00:06<00:00, 14.81it/s]

Run:
phy template-gui  /media/storage/spikesorting_output/SONG/Phy/SC-SONG/phy-params_new/params.py


## compare all output

In [24]:
#comp = sc.compare_two_sorters(sorting_HS, sorting_SC, sorting1_name='HS', sorting2_name='SC')
#sw.plot_agreement_matrix(comp)

#comp = sc.compare_two_sorters(sorting_HS, sorting_TDC, sorting1_name='HS', sorting2_name='TDC')
#sw.plot_agreement_matrix(comp)

comp = si.compare_two_sorters(SC, TDC, sorting1_name='SC', sorting2_name='TDC')
si.plot_agreement_matrix(comp)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
comp.match_event_count

,0,1,2,3,4,5
0,21,40,23,35,25,11
1,4,2,1,2,7,5
10,2,8,5,6,3,4
100,3,8,5,12,5,9
101,10,10,15,11,9,8
...,...,...,...,...,...,...
95,6,18,4,5,5,5
96,3,8,9,4,7,6
97,1,6,0,2,2,5
98,30,54,7,35,20,19


In [26]:
comp.agreement_scores

,0,1,2,3,4,5
0,0.006703,0.009412,0.011564,0.009946,0.008890,0.004077
1,0.002455,0.000723,0.002041,0.000985,0.005348,0.004227
10,0.001181,0.002833,0.009107,0.002871,0.002180,0.003208
100,0.001728,0.002790,0.008446,0.005642,0.003529,0.007004
101,0.004558,0.003003,0.014327,0.004242,0.004792,0.004569
...,...,...,...,...,...,...
95,0.003411,0.006243,0.006462,0.002315,0.003465,0.003802
96,0.001803,0.002862,0.017442,0.001939,0.005212,0.004934
97,0.000655,0.002257,0.000000,0.001038,0.001654,0.004638
98,0.007870,0.010967,0.002599,0.008319,0.005706,0.005625


## some visualizations - TDC

In [27]:
si.plot_rasters(TDC)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
TDC_units_ID = TDC.unit_ids[:] # I couldnt use just unit 2, was giving me an into has not length error (see below)
si.plot_unit_waveforms(we_TDC, 
                       #channel_ids=TDC_units_ID
                       max_channels=12, 
                       #radius_um=60
                      )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
si.plot_isi_distribution(TDC, 
                         bins=100, 
                         window=1
                        )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
si.get_unit_colors(TDC)

In [29]:
colors = {
    0: 'black',
    1: 'green',
    2: 'red',
    3: 'blue',
    4: 'yellow',
    5: 'grey',
    6: 'purple',
    7: 'orange',
    8: 'lightblue'}
w_depth = si.plot_units_depth_vs_amplitude(we_TDC,
                                          unit_colors=colors
                                          )
w_depth.ax.set_aspect(0.1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
si.plot_amplitudes_distribution(we_TDC)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

 ## some visualizations - SC (heavier so won't plot everything) 

In [32]:
w_depth_SC = si.plot_units_depth_vs_amplitude(we_SC)
w_depth_SC.ax.set_aspect(0.1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …